# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [186]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages (0.22.1)


In [187]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import pandas as pd
import re
import numpy as np

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [188]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', con=engine)
X = df["message"]
Y = df[['related', 'request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report']]

In [189]:
print(X.shape)
print(Y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [190]:
def tokenize(text):
    
    # 1. Normalize
    text = text.lower()
    
    # 2. Punctuation Removal
    text = re.sub(r"[^a-zA-Z0-0]", " ", text)
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stopword Removal
    tokens_without_stop = [w for w in tokens if w not in stopwords.words("english")]
    
    # 5. Lemmatizer and strip
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for w in tokens_without_stop:
        clean_tokens.append(lemmatizer.lemmatize(w).strip())
        
    return clean_tokens
    

In [191]:
print(X[9])
print(tokenize(X[9]))

I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )
['croix', 'de', 'bouquet', 'health', 'issue', 'worker', 'santo', 'area', 'croix', 'de', 'bouquet']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [193]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [195]:
from sklearn.metrics import classification_report


In [196]:
preds = pipeline.predict(X_test)

In [197]:
def get_score(test_y, pred_y):
    f1 =[]
    for i in range(test_y.shape[1]):
        print('################   ' + test_y.columns[i] + '   ##################')
        print(classification_report(test_y.iloc[:,i], pred_y[:,i]))
        f1.append(f1_score(test_y.iloc[:,i], pred_y[:,i], average='micro'))
        
    print("The Mean score of F1 is: {}".format(pd.Series(f1).mean()))
    
    return pd.Series(f1).mean()

In [198]:
get_score(y_test, preds)

################   related   ##################
             precision    recall  f1-score   support

          0       0.63      0.43      0.51      1873
          1       0.83      0.92      0.87      5934
          2       0.33      0.40      0.36        58

avg / total       0.78      0.80      0.78      7865

################   request   ##################
             precision    recall  f1-score   support

          0       0.90      0.98      0.93      6533
          1       0.79      0.45      0.57      1332

avg / total       0.88      0.89      0.87      7865

################   offer   ##################
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

################   aid_related   ##################
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4646
 


################   other_weather   ##################
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      7450
          1       0.53      0.06      0.11       415

avg / total       0.93      0.95      0.93      7865

################   direct_report   ##################
             precision    recall  f1-score   support

          0       0.85      0.97      0.91      6321
          1       0.74      0.32      0.44      1544

avg / total       0.83      0.84      0.82      7865

The Mean score of F1 is: 0.9447340538249629


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.


0.94473405382496289

### 6. Improve your model
Use grid search to find better parameters. 

In [199]:
pipeline.get_params(deep=True)

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0f1a846488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [226]:
parameters = {
    'clf__estimator__n_estimators': [1, 2, 3],
}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_macro', n_jobs=4, verbose=2)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=1 ..................................
[CV] clf__estimator__n_estimators=1 ..................................
[CV] clf__estimator__n_estimators=1 ..................................
[CV] clf__estimator__n_estimators=2 ..................................


In [ ]:
preds_tuned = cv.predict(X_test)

In [ ]:
get_score(y_test, preds_tuned)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.